In [43]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torch.utils.data
import pickle

In [44]:

class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.fc1 = nn.Linear(5,30)
        self.fc2 = nn.Linear(30,50)
        self.fc3 = nn.Linear(50,90)
        self.fc4 = nn.Linear(90,120)
        self.fc5 = nn.Linear(120,80)
        self.fc6 = nn.Linear(80,60)
        self.fc7 = nn.Linear(60,30)
        self.fc8 = nn.Linear(30,20)
        self.fc9 = nn.Linear(20,10)
        self.fc10 = nn.Linear(10,1)
# Final output: predicing # of possible accidents

    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = F.dropout(out, p=0.5)
        out = F.relu(self.fc2(out))
        out = F.dropout(out, p=0.5)
        out = F.relu(self.fc3(out))
        out = F.dropout(out, p=0.5)
        out = F.relu(self.fc4(out))
        out = F.dropout(out, p=0.5)
        out = F.relu(self.fc5(out))
        out = F.dropout(out, p=0.5)
        out = F.relu(self.fc6(out))
        out = F.dropout(out, p=0.5)
        out = F.relu(self.fc7(out))
        out = F.dropout(out, p=0.5)
        out = F.relu(self.fc8(out))
        out = F.dropout(out, p=0.5)
        out = self.fc9(out)
        return out

In [45]:
bike = model() 

In [46]:
bike.load_state_dict(torch.load('Model_optimal.pth'))

In [47]:
bike

model(
  (fc1): Linear(in_features=5, out_features=30, bias=True)
  (fc2): Linear(in_features=30, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=90, bias=True)
  (fc4): Linear(in_features=90, out_features=120, bias=True)
  (fc5): Linear(in_features=120, out_features=80, bias=True)
  (fc6): Linear(in_features=80, out_features=60, bias=True)
  (fc7): Linear(in_features=60, out_features=30, bias=True)
  (fc8): Linear(in_features=30, out_features=20, bias=True)
  (fc9): Linear(in_features=20, out_features=10, bias=True)
  (fc10): Linear(in_features=10, out_features=1, bias=True)
)

# Codes

# Time
* 3 = PM
* 1 = RUSH HOUR
* 2 = AM 

Time is the THIRD feature you feed into the model.

In [48]:
data= torch.tensor((2006,1,1,45.5017, -73.5673))

data.shape
output = bike(data)


In [49]:
print(output.shape)

torch.Size([10])


In [50]:
output

tensor([ 3.6829e+16,  3.5253e+16, -3.5292e+16, -3.5597e+16, -3.6441e+16,
        -3.5806e+16, -3.4564e+16, -3.5545e+16, -3.5968e+16, -3.6189e+16])

In [51]:
# 1 Point: (45.501689, -73.567256), 1, 1

data = torch.tensor((2006,1,1,45.501689,-73.567256))
output = bike(data)
output 
# Prediction: 0 -- safe 

tensor([ 3.6829e+16,  3.5253e+16, -3.5292e+16, -3.5597e+16, -3.6441e+16,
        -3.5806e+16, -3.4564e+16, -3.5545e+16, -3.5968e+16, -3.6189e+16])

In [52]:
# 2 Point (45.501689, -73.567256), 1, 2
data = torch.tensor((2006,1,2,45.501689,-73.567256))
output = bike(data)
output 
# Prediction: 0 -- safe

tensor([ 3.6812e+16,  3.5237e+16, -3.5276e+16, -3.5580e+16, -3.6424e+16,
        -3.5789e+16, -3.4548e+16, -3.5528e+16, -3.5952e+16, -3.6172e+16])

In [53]:
# 3 Point (45.501689, -73.567256), 1, 3
data = torch.tensor((2006,1,3,45.501689,-73.567256))
output = bike(data)
output 
# Prediction: 0 -- safe 

tensor([ 3.6795e+16,  3.5221e+16, -3.5259e+16, -3.5564e+16, -3.6407e+16,
        -3.5773e+16, -3.4532e+16, -3.5512e+16, -3.5935e+16, -3.6156e+16])

In [54]:
# 4 Point (45.501689, -73.567256), 2, 1
data = torch.tensor((2006,2,1,45.501689,-73.567256))
output = bike(data)
output 
# Prediction: 0 -- safe

tensor([ 3.6812e+16,  3.5238e+16, -3.5276e+16, -3.5581e+16, -3.6424e+16,
        -3.5790e+16, -3.4548e+16, -3.5529e+16, -3.5952e+16, -3.6173e+16])

In [56]:
# 5 Point (45.501689, -73.567256), 2, 2
data = torch.tensor((2006,2,2,45.501689,-73.567256))
output = bike(data)
output 
# Prediction: 0 -- safe

tensor([ 3.6795e+16,  3.5221e+16, -3.5260e+16, -3.5564e+16, -3.6407e+16,
        -3.5773e+16, -3.4532e+16, -3.5512e+16, -3.5935e+16, -3.6156e+16])

In [69]:
F.softmax(output, dim=0)

tensor([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [57]:
# 6 Point (45.501689, -73.567256), 2, 3
data = torch.tensor((2006,2,3,45.501689,-73.567256))
output = bike(data)
output 
# Prediction: 0 -- safe

tensor([ 3.6778e+16,  3.5205e+16, -3.5243e+16, -3.5548e+16, -3.6391e+16,
        -3.5757e+16, -3.4516e+16, -3.5496e+16, -3.5919e+16, -3.6139e+16])

In [59]:
# 7 Point (45.501689, -73.567256), 5, 3
data = torch.tensor((2006,5,3,45.501689,-73.567256))
output = bike(data)
output 
# Prediction: 0 -- safe

tensor([ 3.6729e+16,  3.5157e+16, -3.5196e+16, -3.5500e+16, -3.6341e+16,
        -3.5708e+16, -3.4470e+16, -3.5448e+16, -3.5870e+16, -3.6090e+16])

In [60]:
# 8 Point (45.501689, -73.567256), 6, 3
data = torch.tensor((2006,6,3,45.501689,-73.567256))
output = bike(data)
output 
# Prediction: 0 -- safe

tensor([ 3.6712e+16,  3.5142e+16, -3.5180e+16, -3.5484e+16, -3.6325e+16,
        -3.5692e+16, -3.4454e+16, -3.5432e+16, -3.5854e+16, -3.6074e+16])

In [61]:
# 9 Point (45.517843, -73.567029), 6, 1
data = torch.tensor((2006,6,1,45.517843,-73.567029))
output = bike(data)
output 
# Prediction: 0 -- safe 

tensor([ 3.6746e+16,  3.5174e+16, -3.5212e+16, -3.5517e+16, -3.6358e+16,
        -3.5725e+16, -3.4486e+16, -3.5465e+16, -3.5887e+16, -3.6107e+16])

In [62]:
# 10 Point (45.512998, -73.569139), 6, 1
data = torch.tensor((2006,6,1,45.512998,-73.569139))
output = bike(data)
output
# Prediction: 0 -- safe

tensor([ 3.6746e+16,  3.5174e+16, -3.5212e+16, -3.5517e+16, -3.6359e+16,
        -3.5725e+16, -3.4486e+16, -3.5465e+16, -3.5887e+16, -3.6108e+16])

In [63]:
# 11 Point (45.511558, -73.578207), 6,1
data = torch.tensor((2006,6,1,45.511558,-73.578207))
output = bike(data)
output
# Prediction: 0 -- safe 

tensor([ 3.6746e+16,  3.5174e+16, -3.5212e+16, -3.5517e+16, -3.6359e+16,
        -3.5725e+16, -3.4486e+16, -3.5465e+16, -3.5887e+16, -3.6108e+16])

In [64]:
# 12 Point (45.511558, -73.578207), 6,2
data = torch.tensor((2006,6,2,45.511558,-73.578207))
output = bike(data)
output
# Prediction: 0 -- safe 

tensor([ 3.6729e+16,  3.5158e+16, -3.5196e+16, -3.5500e+16, -3.6342e+16,
        -3.5709e+16, -3.4470e+16, -3.5448e+16, -3.5871e+16, -3.6091e+16])

In [65]:
# 13 Point (45.511558, -73.578207), 6,3
data = torch.tensor((2006,6,3,45.511558,-73.578207))
output = bike(data)
output
# Prediction: 0 -- safe 

tensor([ 3.6712e+16,  3.5142e+16, -3.5180e+16, -3.5484e+16, -3.6325e+16,
        -3.5692e+16, -3.4454e+16, -3.5432e+16, -3.5854e+16, -3.6074e+16])

In [66]:
# 14 Point (45.512095, -73.568483), 7, 1
data = torch.tensor((2006,7,1,45.511558,-73.578207))
output = bike(data)
output
# Prediction: 0 -- safe 

tensor([ 3.6729e+16,  3.5158e+16, -3.5197e+16, -3.5501e+16, -3.6342e+16,
        -3.5709e+16, -3.4471e+16, -3.5449e+16, -3.5871e+16, -3.6091e+16])